In [14]:
import os
import requests
from dotenv import load_dotenv
from langchain.chains import ConversationalRetrievalChain
from langchain.memory import ConversationBufferMemory
# Changed import for Groq
from langchain_groq import ChatGroq
from langchain_core.prompts import PromptTemplate
from langchain.agents import AgentExecutor, Tool, create_react_agent
from langchain_core.messages import AIMessage, HumanMessage
# Import for SerperDevTool
from langchain_community.utilities import GoogleSerperAPIWrapper

In [15]:
load_dotenv()

True

In [16]:
# --- Configuration (User needs to fill these in their environment) ---
# Load API keys from environment variables
GROQ_API_KEY = os.getenv("GROQ_API_KEY")
OPENWEATHER_API_KEY = os.getenv("OPENWEATHER_API_KEY")
SERPER_API_KEY = os.getenv("SERPER_API_KEY") # New API Key for Serper

In [17]:
# Llama Model ID (from Groq)
LLAMA_MODEL_ID = "meta-llama/llama-4-scout-17b-16e-instruct" # A common Llama model on Groq, you can change this if needed

In [18]:
# --- Weather Tool Definition ---
def get_current_weather(location: str) -> str:
    """
    Fetches the current weather conditions for a given location using the OpenWeatherMap API.

    Args:
        location (str): The city name for which to get the weather.

    Returns:
        str: A string describing the current weather, or an error message.
    """
    if not OPENWEATHER_API_KEY:
        return "OpenWeatherMap API key is not configured. Please set the OPENWEATHER_API_KEY environment variable."

    base_url = "http://api.openweathermap.org/data/2.5/weather"
    params = {
        "q": location,
        "appid": OPENWEATHER_API_KEY,
        "units": "metric" # You can change to 'imperial' for Fahrenheit
    }
    try:
        response = requests.get(base_url, params=params)
        response.raise_for_status() # Raise an HTTPError for bad responses (4xx or 5xx)
        data = response.json()

        if data.get("cod") == 200:
            main_data = data.get("main", {})
            weather_data = data.get("weather", [{}])[0]
            wind_data = data.get("wind", {})

            temperature = main_data.get("temp")
            feels_like = main_data.get("feels_like")
            humidity = main_data.get("humidity")
            description = weather_data.get("description", "N/A")
            city_name = data.get("name", location)
            wind_speed = wind_data.get("speed")

            return (
                f"Current weather in {city_name}:\n"
                f"Temperature: {temperature}°C (feels like {feels_like}°C)\n"
                f"Description: {description}\n"
                f"Humidity: {humidity}%\n"
                f"Wind speed: {wind_speed} m/s"
            )
        else:
            return f"Could not retrieve weather for {location}. Error: {data.get('message', 'Unknown error')}"
    except requests.exceptions.RequestException as e:
        return f"Error connecting to OpenWeatherMap API: {e}"
    except Exception as e:
        return f"An unexpected error occurred: {e}"

In [19]:
# Define the Langchain Tool
weather_tool = Tool(
    name="get_current_weather",
    func=get_current_weather,
    description="Useful for getting the current weather in a specified location. Input should be a city name.",
)

In [22]:
# --- NEW: Google Search Tool Definition (Corrected) ---
# Initialize the GoogleSerperAPIWrapper
serper_search = GoogleSerperAPIWrapper()

# Create a Langchain Tool from the GoogleSerperAPIWrapper
search_tool = Tool(
    name="Google Search",
    func=serper_search.run, # Use the .run() method of the wrapper
    description="Useful for answering general knowledge questions, current events, or anything that requires searching the internet. Input should be a search query.",
)

In [23]:
# --- Chatbot Setup ---

# Initialize the Llama model using ChatGroq
try:
    if not GROQ_API_KEY:
        raise ValueError("GROQ_API_KEY environment variable is not set. Please configure it.")

    llm = ChatGroq(
        temperature=0.7,
        model_name=LLAMA_MODEL_ID,
        groq_api_key=GROQ_API_KEY
    )
except ValueError as ve:
    print(f"Configuration Error: {ve}")
    print("Using a dummy LLM for demonstration. Please ensure GROQ_API_KEY is set in your environment.")
    # Fallback to a dummy LLM for demonstration if Groq setup fails
    from langchain.llms.fake import FakeListLLM
    llm = FakeListLLM(responses=["Hello, how can I help you?", "The weather is sunny.", "I remember that.", "I can search for that information."])
except Exception as e:
    print(f"An unexpected error occurred during LLM initialization: {e}")
    print("Using a dummy LLM for demonstration. Please check your Groq configuration.")
    from langchain.llms.fake import FakeListLLM
    llm = FakeListLLM(responses=["Hello, how can I help you?", "The weather is sunny.", "I remember that.", "I can search for that information."])

In [24]:
# Initialize memory for the chatbot
memory = ConversationBufferMemory(memory_key="chat_history", return_messages=True)

# Define the prompt for the agent
# The prompt instructs the agent on how to use the tools and respond.
# It also includes placeholders for the chat history and user input.
prompt = PromptTemplate.from_template("""
You are a helpful AI assistant. You have access to the following tools:
{tools}

Use the following format:

Question: the input question you must answer
Thought: you should always think about what to do
Action: the action to take, should be one of [{tool_names}]
Action Input: the input to the action
Observation: the result of the action
... (this Thought/Action/Action Input/Observation can repeat N times)
Thought: I now know the final answer
Final Answer: the final answer to the original input question

Begin!

{chat_history}
Question: {input}
{agent_scratchpad}
""")

In [25]:
# Create the agent - NOW WITH MULTIPLE TOOLS!
agent = create_react_agent(llm, [weather_tool, search_tool], prompt) # Add search_tool here

# Create the AgentExecutor - NOW WITH MULTIPLE TOOLS!
agent_executor = AgentExecutor(
    agent=agent,
    tools=[weather_tool, search_tool], # Add search_tool here
    verbose=True, # Set to True to see the agent's thought process
    memory=memory,
    handle_parsing_errors=True # Robustly handle potential parsing errors from the LLM
)

In [ ]:
# --- Chatbot Interaction Loop ---
def chat_with_bot():
    print("Welcome to the Chatbot! Type 'exit' to end the conversation.")
    print("Try asking about the weather, e.g., 'What's the weather in London?'")
    print("Or try asking general questions, e.g., 'Who won the last Super Bowl?' or 'What is the capital of Canada?'")

    while True:
        user_input = input("You: ")
        if user_input.lower() == 'exit':
            print("Chatbot: Goodbye!")
            break

        try:
            # The agent_executor handles both memory and tool usage
            response = agent_executor.invoke({"input": user_input})
            print(f"Chatbot: {response['output']}")
        except Exception as e:
            print(f"Chatbot Error: {e}")
            print("Please try again.")

if __name__ == "__main__":
    chat_with_bot()

Welcome to the Chatbot! Type 'exit' to end the conversation.
Try asking about the weather, e.g., 'What's the weather in London?'
Or try asking general questions, e.g., 'Who won the last Super Bowl?' or 'What is the capital of Canada?'


You:  WHat is the weather in Vijayawada?




> Entering new AgentExecutor chain...
Thought: To find out the current weather in Vijayawada, I should use the get_current_weather tool.

Action: get_current_weather
Action Input: VijayawadaCurrent weather in Vijayawada:
Temperature: 34.91°C (feels like 37.98°C)
Description: overcast clouds
Humidity: 43%
It seems I have all the information I need.

Thought: I now know the final answer
Final Answer: The current weather in Vijayawada is:
Temperature: 34.91°C (feels like 37.98°C)
Description: overcast clouds
Humidity: 43%
Wind speed: 6.96 m/s

> Finished chain.
Chatbot: The current weather in Vijayawada is:
Temperature: 34.91°C (feels like 37.98°C)
Description: overcast clouds
Humidity: 43%
Wind speed: 6.96 m/s
